In [ ]:
#ChromeOS
sudo apt install -y autoconf libaio-dev libbluetooth-dev libbrlapi-dev libbz2-dev libcap-dev libcap-ng-dev libcurl4-gnutls-dev libepoxy-dev libfdt-dev libgbm-dev libgles2-mesa-dev libglib2.0-dev libgtk-3-dev libibverbs-dev libjpeg8-dev liblzo2-dev libncurses5-dev libnuma-dev librbd-dev librdmacm-dev libsasl2-dev libsdl1.2-dev libsdl2-dev libseccomp-dev libsnappy-dev libssh2-1-dev libspice-server-dev libspice-server1 libtool libusb-1.0-0 libusb-1.0-0-dev libvde-dev libvdeplug-dev libvte-dev libxen-dev valgrind xfslibs-dev xutils-dev zlib1g-dev libusbredirhost-dev usbredirserver

#virglrenderer
git clone git://git.freedesktop.org/git/virglrenderer
cd virglrenderer
./autogen.sh
make -j7
sudo make install

#https://unix.stackexchange.com/questions/268460/how-to-mount-qcow2-image
#Step 1 - Enable NBD on the host
sudo modprobe nbd max_part=8
#Step 2 - Connect the QCOW2 as a network block device
sudo qemu-nbd --connect=/dev/nbd0 /var/lib/libvirt/images/ChromeOS-1.qcow2
#Step 3 - List partitions inside the QCOW2
sudo fdisk /dev/nbd0 -l
#Step 4 - Mount the partition from the VM
#sudo mount /dev/nbd0p1 /mnt/somepoint/


#COS
sudo apt-get update
sudo apt-get install -y figlet 
sudo apt-get install -y pv
sudo apt-get install -y cgpt
cd /media/$USER/Ventoy/ChromeOS
sudo bash chromeos-install.sh -src rammus_recovery.bin -dst /dev/nbd0

#ATBVanilla
sudo dd if=/home/$USER/Documents/chromiumos_image.img of=/dev/nbd0 bs=4M

#Step 5 - After you're done, unmount and disconnect
#sudo umount /mnt/somepoint/
sudo qemu-nbd --disconnect /dev/nbd0
sudo rmmod nbd


cd /media/user/Ventoy/ChromeOS
sudo /usr/bin/qemu-system-x86_64 \
	-bios /usr/share/OVMF/OVMF_CODE.fd \
    -enable-kvm \
    -m 2G \
    -smp 4 \
    -drive id=COS,if=none,snapshot=on,format=qcow2,file='/var/lib/libvirt/images/ChromeOS-1.qcow2' \
    -vga virtio \
    -net nic,model=virtio \
    -net user,hostfwd=tcp:127.0.0.1:9222-:22 \
    -usb -usbdevice keyboard \
    -usbdevice mouse \
    -device virtio-gpu-pci,virgl \
    -display gtk,gl=on

#https://forum.manjaro.org/t/qemu-kvm-vm-in-virt-manager-is-running-with-a-black-screen-when-opengl-acceleration-is-turned-on/138632/9
sudo qemu-system-x86_64 -name COS -drive file=/var/lib/libvirt/images/ChromeOS-1.qcow2 -m 8192 -enable-kvm -M q35 -cpu host -smp 4 -display egl-headless,gl=on -usb -device usb-tablet -bios /usr/share/OVMF/OVMF_CODE.fd -boot order=c -device virtio-vga,virgl=on -spice unix,disable-ticketing,image-compression=off,addr=cos.sock,seamless-migration=on

sudo remote-viewer spice+unix://cos.sock


In [ ]:
wget http://launchpadlibrarian.net/395859364/python-kerberos_1.1.14-1build2_amd64.deb
wget http://launchpadlibrarian.net/406841884/repo_1.13.2-1_all.deb
sudo apt-get install python3-kerberos
#https://chromium.arnoldthebat.co.uk/index.php?dir=special&order=modified&sort=desc
#https://github.com/imperador/chromefy
#https://forum.xda-developers.com/hardware-hacking/chromebooks/guide-installing-official-chrome-os-pc-t3865697
#https://www.collabora.com/news-and-blog/blog/2017/12/01/building-chromiumos-for-qemu/
#https://github.com/FydeOS/chromium_os-raspberry_pi
#https://github.com/FydeOS/chromium_os-vm-vmware

wget https://download.dm.origin.com/origin/live/OriginThinSetup.exe